So it's not possible to get the method ID during a token transfer and while I can get it from the standard tx list, the token transfers are not part of that list.

The easiest way is probable just to take the transfers from the Sickle contract to my wallet and check to make sure no value is above a certain amount, like $5k or something, which will be an exit and not rebalance dust or harvesting.  

To do this right, I would need to get the tx hashes and then look them up and figure out what the method is for each of them.
- This turns out to fail a lot.  It could be the web3 function I'm using.  Not sure it's useful to debug.

In [ ]:
import transaction_class as tc
import private

# Method IDs
rebalance_for = '0xe6fb317f'
harvest = '0x5ec5999e'
wallet = private.wal_lp
sender = private.sickle_lp
# start_block = 25707613  # Use the block after the last deposit
start_block = 25915461  # Start fresh on Feb 03 5pm

wal = tc.Transactions(wallet)
wal.get_transactions(start_block)
wal.filter_tx_list_by_sender(sender)
wal.create_dataframe()
wal.get_usd_prices()
wal.update_prices()
df = wal.df

In [ ]:
df.pivot_table(index='tokenSymbol', values=['value', 'valueUsd'], aggfunc='sum')

In [3]:
df.to_csv('vfat_lp_dust_transactions.csv')